In [1]:

# ================================================
# 1️⃣ Env Prepare Install Required Packages
# ================================================

#%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo 

Using Python 3.12.12 environment at: /home/ospost/micromamba/envs/ML
Resolved 5 packages in 13ms                                          
Audited 5 packages in 0.16ms


In [2]:
import torch
import bitsandbytes as bnb
from collections import Counter
from unsloth import FastLanguageModel


model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/gpt-oss-20b",
    load_in_4bit=True,
)

bnb_layers = []
for name, module in model.named_modules():
    if isinstance(module, bnb.nn.Linear4bit):
        bnb_layers.append(name)

print(f"BNB 4-bit layers found: {len(bnb_layers)}")
print("Sample BNB layer:", bnb_layers[0] if bnb_layers else "❌ NONE")
print("Model compute dtype:", model.dtype)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ospost/micromamba/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.2: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    GRID A100D-40C. Num GPUs = 1. Max memory: 39.996 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1. CUDA: 8.0. CUDA Toolkit: 12.9. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.21s/it]


BNB 4-bit layers found: 1632
Sample BNB layer: model.layers.0.self_attn.q_proj
Model compute dtype: torch.bfloat16


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    checkpoint_path="ospost/gpt-oss-20b-sft-adapter"
)


Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
assert any("lora_" in n for n,_ in model.named_parameters())

lora_params = [
    (n, p) for n, p in model.named_parameters() if "lora_" in n
]

print(f"LoRA parameters found: {len(lora_params)}")
print("Sample LoRA param:", lora_params[0][0] if lora_params else "❌ NONE")

print("LoRA dtypes:", Counter(p.dtype for _, p in lora_params))

trainable = [
    n for n, p in model.named_parameters() if p.requires_grad
]

print(f"Trainable params count: {len(trainable)}")
print("Sample trainable param:", trainable[0] if trainable else "❌ NONE")



LoRA parameters found: 192
Sample LoRA param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
LoRA dtypes: Counter({torch.float32: 192})
Trainable params count: 192
Sample trainable param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight


In [5]:
model.save_pretrained_merged(
    "gpt-oss-20b-sft-mxfp4",
    tokenizer,
    save_method="mxfp4",
)


Found HuggingFace hub cache directory: /home/ospost/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.50s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into mxfp4: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.84s/it]


Unsloth: Merge process complete. Saved to `/home/ospost/workspace/gpt-oss-20b-fine-tune/gpt-oss-20b-sft-mxfp4`


In [6]:
!ls -lah gpt-oss-20b-sft-mxfp4

total 13G
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 14:07 .
drwxrwxr-x 7 ospost ospost 4.0K Jan  9 14:07 ..
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 14:07 .cache
-rw-rw-r-- 1 ospost ospost  15K Jan  9 14:07 chat_template.jinja
-rw-rw-r-- 1 ospost ospost 1.8K Jan  9 14:07 config.json
-rw-rw-r-- 1 ospost ospost 4.5G Jan  9 14:07 model-00000-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost 4.5G Jan  9 14:07 model-00001-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost 3.9G Jan  9 14:07 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost  36K Jan  9 14:07 model.safetensors.index.json
-rw-rw-r-- 1 ospost ospost  446 Jan  9 14:07 special_tokens_map.json
-rw-rw-r-- 1 ospost ospost  20K Jan  9 14:07 tokenizer_config.json
-rw-rw-r-- 1 ospost ospost  27M Jan  9 14:07 tokenizer.json


In [7]:
model.save_pretrained_merged(
    "gpt-oss-20b-sft-bf16",
    tokenizer
)


Found HuggingFace hub cache directory: /home/ospost/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.69s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [02:20<00:00, 46.99s/it]


Unsloth: Regenerating safetensors index for dequantized MXFP4 model...
Unsloth: Merge process complete. Saved to `/home/ospost/workspace/gpt-oss-20b-fine-tune/gpt-oss-20b-sft-bf16`


In [8]:
!ls -lah gpt-oss-20b-sft-bf16

total 39G
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 14:10 .
drwxrwxr-x 8 ospost ospost 4.0K Jan  9 14:08 ..
drwxrwxr-x 3 ospost ospost 4.0K Jan  9 14:07 .cache
-rw-rw-r-- 1 ospost ospost  15K Jan  9 14:07 chat_template.jinja
-rw-rw-r-- 1 ospost ospost 1.9K Jan  9 14:07 config.json
-rw------- 1 ospost ospost  16G Jan  9 14:08 model-00000-of-00002.safetensors
-rw------- 1 ospost ospost  16G Jan  9 14:09 model-00001-of-00002.safetensors
-rw------- 1 ospost ospost 8.3G Jan  9 14:10 model-00002-of-00002.safetensors
-rw-rw-r-- 1 ospost ospost  35K Jan  9 14:10 model.safetensors.index.json
-rw-rw-r-- 1 ospost ospost  446 Jan  9 14:07 special_tokens_map.json
-rw-rw-r-- 1 ospost ospost  20K Jan  9 14:07 tokenizer_config.json
-rw-rw-r-- 1 ospost ospost  27M Jan  9 14:07 tokenizer.json


In [2]:
import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "gpt-oss-20b-sft-bf16",
    torch_dtype="auto",
    device_map="auto"  # or "auto"
)

next(model.parameters()).dtype


/home/ospost/micromamba/envs/ML/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.06s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


torch.bfloat16

In [ ]:
# model.push_to_hub("ospost/gpt-oss-20b-sft-mxfp4", token = "")


In [ ]:
# git clone https://github.com/ggerganov/llama.cpp
# cd llama.cpp

# python convert_hf_to_gguf.py \
#  gpt-oss-20b-sft-mxfp4 \
#  --outtype q4_K_M \
#  --outfile gpt-oss-20b-sft-q4_K_M.gguf
